# EN3160 Assignment 3 on Neural Networks

Instructed by Dr. Ranga Rodrigo

Done by Jayakumar W.S. (210236P)

### Introduction

This assignment is focused on implementing neural networks for image classification. This is done by using:
1. Our own neural network implementation
2. An implementation of LeNet-5
3. An implementation of ResNet-18

### Import necessary libraries

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchinfo import summary
import matplotlib.pyplot as plt
import gc

### Dataloading

In [ ]:
transform = transforms.Compose ([ transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5) , (0.5, 0.5, 0.5))])
batch_size = 32
trainset = torchvision.datasets.CIFAR10(root= './data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root= './data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device begin used : {device}")

### Our own architecture

#### Define Network Parameters

In [ ]:
Din = 3*32*32 # Input size (flattened CIFAR=10 image size)
K = 10 # Output size (number of classes in CIFAR=10)
std = 1e-5
# Initialize weights and biases
w = torch.randn(Din, K, device=device, dtype=torch.float, requires_grad=True) * std
b = torch.randn(K, device=device, dtype=torch.float, requires_grad=True)
# Hyperparameters
iterations = 20
lr = 2e-6 # Learning rate
lr_decay = 0.9 # Learning rate decay
reg = 0 # Regularization
loss_history = [ ]

In [ ]:
for t in range(iterations):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get inputs and labels
        inputs, labels = data
        Ntr = inputs.shape[0]  # Batch size
        x_train = inputs.view(Ntr, -1).to(device)  # Flatten input to (Ntr, Din)
        y_train_onehot = nn.functional.one_hot(labels, K).float().to(device)  # Convert labels to one-hot

        # Forward pass
        y_pred = x_train.mm(w) + b  # Output layer activation

        # Loss calculation (Mean Squared Error with regularization)
        loss = (1/Ntr) * torch.sum((y_pred - y_train_onehot) ** 2) + reg * torch.sum(w ** 2)
        loss_history.append(loss.item())
        running_loss += loss.item()

        # Backpropagation
        dy_pred = (2.0 / Ntr) * (y_pred - y_train_onehot)
        dw = x_train.t().mm(dy_pred) + reg * w
        db = dy_pred.sum(dim=0)

        # Parameter update
        w = w - lr * dw
        b = b - lr * db

    print(f"Epoch {t + 1} / {iterations}, Loss: {running_loss / len(trainloader)}")

    # Learning rate decay
    lr *= lr_decay

In [ ]:
del w, b, x_train, y_train_onehot, y_pred, loss, dy_pred, dw, db
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# This implementation is not efficient and is only for educational purposes. For real-world applications, use PyTorch's built-in functions and classes. This fails
# as memory usage increases with the number of iterations.

Din = 3*32*32 # Input size (flattened CIFAR=10 image size)
K = 10 # Output size (number of classes in CIFAR=10)
std = 1e-5
# Initialize weights and biases
w1 = torch.randn(Din, 100, device=device, requires_grad=True)
b1 = torch.zeros(100, device=device, requires_grad=True)
w2 = torch.randn(100, K, device=device, requires_grad=True)
b2 = torch.zeros(K, device=device, requires_grad=True)
# Hyperparameters
iterations = 20
lr = 2e-6 # Learning rate
lr_decay = 0.9 # Learning rate decay
reg = 0 # Regularization
loss_history = [ ]

#### Training loop

In [ ]:
for t in range(iterations) :
    running_loss = 0.0
    for i , data in enumerate(trainloader, 0) :
        # Get inputs and labe l s
        inputs , labels = data
        Ntr = inputs.shape[0] # Batch size
        x_train = inputs.view(Ntr, -1).to(device) # Flatten input to (Ntr, Din)
        y_train_onehot = nn.functional.one_hot(labels, K).float().to(device) # Convert labe l s to one=hot # Forward pass
        hidden = x_train.mm(w1) + b1
        y_pred = hidden.mm(w2) + b2
        # Loss calculation (Mean Squared Error with regularization)
        loss = (1/Ntr) * torch.sum((y_pred - y_train_onehot) ** 2) + reg * (torch.sum(w1 ** 2) + torch.sum(w2 ** 2))
        loss_history.append(loss.item())
        running_loss += loss.item()
        # Backpropagation
        dy_pred = (2.0 / Ntr) * (y_pred - y_train_onehot)
        dhidden = dy_pred.mm(w2.t()) 
        dw2 = hidden.t().mm(dy_pred) + reg * w2
        db2 = dy_pred.sum(dim=0)
        dw1 = x_train.t().mm(dhidden) + reg * w1
        db1 = dhidden.sum(dim=0)
        # Parameter update
        w2 = w2 - lr * dw2
        b2 = b2 - lr * db2
        w1 = w1 - lr * dw1
        b1 = b1 - lr * db1
    print(f"Epoch {t+1} / {iterations} , Loss : {running_loss/len(trainloader)}")
    # Learning rat e decay
    lr *= lr_decay

In [ ]:
del w1, b1, w2, b2, x_train, y_train_onehot, y_pred, loss, dy_pred, dhidden, dw2, db2, dw1, db1
gc.collect()
torch.cuda.empty_cache()

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, Din, H, Dout):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(Din, H)
        self.linear2 = nn.Linear(H, Dout)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [ ]:
model = NeuralNetwork(Din, 100, K).to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=reg)

In [ ]:
for t in range(iterations):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get inputs and labels
        inputs, labels = data
        Ntr = inputs.shape[0]  # Batch size
        x_train = inputs.view(Ntr, -1).to(device)  # Flatten input to (Ntr, Din)
        y_train = labels.to(device)  # Convert labels to one-hot

        # Forward pass
        y_pred = model(x_train)

        # Loss calculation
        loss_val = loss(y_pred, y_train)
        loss_history.append(loss_val.item())
        running_loss += loss_val.item()

        # Backpropagation
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

    print(f"Epoch {t + 1} / {iterations}, Loss: {running_loss / len(trainloader)}")

In [ ]:
accuracy = 0
model.eval()
with  torch.inference_mode():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        x_test, y_test = inputs.to(device), labels.to(device)
        y_pred = model(x_test)
        _, predicted = torch.max(y_pred, 1)
        accuracy += (predicted == y_test).sum().item()

print(f"Accuracy: {accuracy / len(testset)}")

### LeNet-5

In [3]:
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
classes = tuple(str(i) for i in range(10))

In [5]:
class LeNet(nn.Module):
    def __init__(self, input_size, input_channels, output_size):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channels, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        conv_output_size = ((input_size - 4) // 2 - 4) // 2
        self.classifier = nn.Sequential(
            nn.Linear(16 * conv_output_size * conv_output_size, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, output_size)
        )

    def forward(self, x):
        y = self.conv1(x)
        y = self.conv2(y)
        y = y.view(y.size(0), -1)
        y = self.classifier(y)
        return y

In [6]:
model = LeNet(input_size = 28, input_channels = 1, output_size = 10).to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_history = [ ]
iterations = 10

In [7]:
for t in range(iterations):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get inputs and labels
        inputs, labels = data
        x_train, y_train = inputs.to(device), labels.to(device)

        # Forward pass
        y_pred = model(x_train)

        # Loss calculation
        loss_val = loss(y_pred, y_train)
        loss_history.append(loss_val.item())
        running_loss += loss_val.item()

        # Backpropagation
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

    print(f"Epoch {t + 1} / {iterations}, Loss: {running_loss / len(trainloader)}")

Epoch 1 / 10, Loss: 0.22979344264939428
Epoch 2 / 10, Loss: 0.06832440779755512
Epoch 3 / 10, Loss: 0.04808383764217918
Epoch 4 / 10, Loss: 0.03904317057208003
Epoch 5 / 10, Loss: 0.03111796266750122
Epoch 6 / 10, Loss: 0.027499199597144617
Epoch 7 / 10, Loss: 0.02319336009456941
Epoch 8 / 10, Loss: 0.01949250753870971
Epoch 9 / 10, Loss: 0.01712961347642146
Epoch 10 / 10, Loss: 0.014968328931884146


In [8]:
accuracy = 0
model.eval()
with  torch.inference_mode():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        x_test, y_test = inputs.to(device), labels.to(device)
        y_pred = model(x_test)
        _, predicted = torch.max(y_pred, 1)
        accuracy += (predicted == y_test).sum().item()

print(f"Accuracy: {accuracy / len(testset)}")

Accuracy: 0.9897


In [11]:
del model, loss, optimizer, x_train, y_train, y_pred, loss_val
gc.collect()
torch.cuda.empty_cache()

### Implementing ResNet-18

In [52]:
model = torchvision.models.resnet18(weights = 'IMAGENET1K_V1').to(device)
data_folder = './data/hymenoptera_data'
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
trainset = torchvision.datasets.ImageFolder(root=f'{data_folder}/train', transform=train_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_transforms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
testset = torchvision.datasets.ImageFolder(root=f'{data_folder}/val', transform=test_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
classes = trainset.classes

In [53]:
print(summary(model, input_size=(batch_size, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"]))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
ResNet                                   [32, 3, 224, 224]         [32, 1000]                --                        True
├─Conv2d: 1-1                            [32, 3, 224, 224]         [32, 64, 112, 112]        9,408                     True
├─BatchNorm2d: 1-2                       [32, 64, 112, 112]        [32, 64, 112, 112]        128                       True
├─ReLU: 1-3                              [32, 64, 112, 112]        [32, 64, 112, 112]        --                        --
├─MaxPool2d: 1-4                         [32, 64, 112, 112]        [32, 64, 56, 56]          --                        --
├─Sequential: 1-5                        [32, 64, 56, 56]          [32, 64, 56, 56]          --                        True
│    └─BasicBlock: 2-1                   [32, 64, 56, 56]          [32, 64, 56, 56]          --                        True
│    │ 

In [54]:
model.fc = nn.Linear(512, len(classes)).to(device)

In [55]:
print(summary(model, input_size=(batch_size, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"]))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
ResNet                                   [32, 3, 224, 224]         [32, 2]                   --                        True
├─Conv2d: 1-1                            [32, 3, 224, 224]         [32, 64, 112, 112]        9,408                     True
├─BatchNorm2d: 1-2                       [32, 64, 112, 112]        [32, 64, 112, 112]        128                       True
├─ReLU: 1-3                              [32, 64, 112, 112]        [32, 64, 112, 112]        --                        --
├─MaxPool2d: 1-4                         [32, 64, 112, 112]        [32, 64, 56, 56]          --                        --
├─Sequential: 1-5                        [32, 64, 56, 56]          [32, 64, 56, 56]          --                        True
│    └─BasicBlock: 2-1                   [32, 64, 56, 56]          [32, 64, 56, 56]          --                        True
│    │ 

In [56]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_history = []
iterations = 15

In [57]:
for t in range(iterations):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get inputs and labels
        inputs, labels = data
        x_train, y_train = inputs.to(device), labels.to(device)

        # Forward pass
        y_pred = model(x_train)

        # Loss calculation
        loss_val = loss(y_pred, y_train)
        loss_history.append(loss_val.item())
        running_loss += loss_val.item()

        # Backpropagation
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

    print(f"Epoch {t + 1} / {iterations}, Loss: {running_loss / len(trainloader)}")

Epoch 1 / 15, Loss: 0.7344811707735062
Epoch 2 / 15, Loss: 0.7005684748291969
Epoch 3 / 15, Loss: 0.4183211587369442
Epoch 4 / 15, Loss: 0.3508832957595587
Epoch 5 / 15, Loss: 0.31578759104013443
Epoch 6 / 15, Loss: 0.23475292138755322
Epoch 7 / 15, Loss: 0.2736297268420458
Epoch 8 / 15, Loss: 0.21533881919458508
Epoch 9 / 15, Loss: 0.1913422178477049
Epoch 10 / 15, Loss: 0.250522131100297
Epoch 11 / 15, Loss: 0.2673525484278798
Epoch 12 / 15, Loss: 0.23732879757881165
Epoch 13 / 15, Loss: 0.20999886561185122
Epoch 14 / 15, Loss: 0.1920069344341755
Epoch 15 / 15, Loss: 0.2966100051999092


In [59]:
accuracy = 0
model.eval()
with  torch.inference_mode():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        x_test, y_test = inputs.to(device), labels.to(device)
        y_pred = model(x_test)
        _, predicted = torch.max(y_pred, 1)
        accuracy += (predicted == y_test).sum().item()

print(f"Accuracy: {accuracy / len(testset)}")

Accuracy: 0.869281045751634


In [61]:
model = torchvision.models.resnet18(weights = 'IMAGENET1K_V1').to(device)
model.fc = nn.Linear(512, len(classes)).to(device)
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

In [64]:
print(summary(model, input_size=(batch_size, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"]))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
ResNet                                   [32, 3, 224, 224]         [32, 2]                   --                        Partial
├─Conv2d: 1-1                            [32, 3, 224, 224]         [32, 64, 112, 112]        (9,408)                   False
├─BatchNorm2d: 1-2                       [32, 64, 112, 112]        [32, 64, 112, 112]        (128)                     False
├─ReLU: 1-3                              [32, 64, 112, 112]        [32, 64, 112, 112]        --                        --
├─MaxPool2d: 1-4                         [32, 64, 112, 112]        [32, 64, 56, 56]          --                        --
├─Sequential: 1-5                        [32, 64, 56, 56]          [32, 64, 56, 56]          --                        False
│    └─BasicBlock: 2-1                   [32, 64, 56, 56]          [32, 64, 56, 56]          --                        False


In [62]:
for t in range(iterations):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get inputs and labels
        inputs, labels = data
        x_train, y_train = inputs.to(device), labels.to(device)

        # Forward pass
        y_pred = model(x_train)

        # Loss calculation
        loss_val = loss(y_pred, y_train)
        loss_history.append(loss_val.item())
        running_loss += loss_val.item()

        # Backpropagation
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

    print(f"Epoch {t + 1} / {iterations}, Loss: {running_loss / len(trainloader)}")

Epoch 1 / 15, Loss: 0.8798065930604935
Epoch 2 / 15, Loss: 0.8848210275173187
Epoch 3 / 15, Loss: 0.8824147954583168
Epoch 4 / 15, Loss: 0.8799518868327141
Epoch 5 / 15, Loss: 0.8958351314067841
Epoch 6 / 15, Loss: 0.8909963145852089
Epoch 7 / 15, Loss: 0.8773292377591133
Epoch 8 / 15, Loss: 0.9258950874209404
Epoch 9 / 15, Loss: 0.9247441440820694
Epoch 10 / 15, Loss: 0.8841805011034012
Epoch 11 / 15, Loss: 0.9186692908406258
Epoch 12 / 15, Loss: 0.8964697495102882
Epoch 13 / 15, Loss: 0.8738040551543236
Epoch 14 / 15, Loss: 0.8921055868268013
Epoch 15 / 15, Loss: 0.9088046252727509


In [63]:
accuracy = 0
model.eval()
with  torch.inference_mode():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        x_test, y_test = inputs.to(device), labels.to(device)
        y_pred = model(x_test)
        _, predicted = torch.max(y_pred, 1)
        accuracy += (predicted == y_test).sum().item()

print(f"Accuracy: {accuracy / len(testset)}")

Accuracy: 0.26143790849673204
